# Caso Práctico

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## For plotting
import plotly.express as px
import plotly.graph_objects as go
import plotly.tools

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
bops_bm = pd.read_csv('bops_bm.csv')
bops_ol = pd.read_csv('bops_online.csv')

In [10]:
dicc = pd.ExcelFile('diccionarios.xlsx')

In [12]:
bops_bm.shape, bops_ol.shape

((4538, 11), (10710, 17))

In [4]:
bops_bm.head()

,id (store),year,month,week,usa,after,sales,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1.0,2011.0,4.0,16.0,0.0,0.0,"118,691",NaN,NaN,NaN,NaN
1,1.0,2011.0,4.0,17.0,0.0,0.0,"113,804",NaN,NaN,NaN,NaN
2,1.0,2011.0,4.0,18.0,0.0,0.0,"172,104",NaN,NaN,NaN,NaN
3,1.0,2011.0,5.0,19.0,0.0,0.0,"105,591",NaN,NaN,NaN,NaN
4,1.0,2011.0,5.0,20.0,0.0,0.0,"94,884",NaN,NaN,NaN,NaN


In [6]:
bops_ol.head()

,id (DMA),year,month,week,after,close,sales,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1,2011,4,17,0,1,"18,564",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2011,4,18,0,1,"30,883",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2011,5,19,0,1,"37,425",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2011,5,20,0,1,"32,563",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2011,5,21,0,1,"35,773",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
dicc.sheet_names

['Online Sales', 'Online Data Dictionary', 'B&M Sales', 'B&M Data Dictionary']

In [21]:
ol_dd = dicc.parse('Online Data Dictionary')
ol_dd

/Users/anieto/miniconda3/envs/zsessions/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning:

Unknown extension is not supported and will be removed



,id (DMA),Designated Market Area (DMA) identification code
0,year,Year
1,month,Month number (January =1)
2,week,Week number (first week of the year = 1)
3,after,Equal to 1 if week is after introduction of BO...
4,close,Equal to 1 if week is DMA is within 50 miles o...
5,sales,Total online sales for the week for the DMA in...


In [20]:
bm_dd = dicc.parse('B&M Data Dictionary')
bm_dd

/Users/anieto/miniconda3/envs/zsessions/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning:

Unknown extension is not supported and will be removed



,id (store),Store identification code
0,year,Year
1,month,Month (January = 1)
2,week,Week (first week of the year = 1)
3,usa,Equal to 1 if store is in the USA; 0 if it is ...
4,after,Equal to 1 if week is after introduction of BO...
5,sales,Total B&M sales for the week for the store
